In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import X_test and y_test
X_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\X_test.csv')
y_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\y_test.csv')

# 1st column in these dataets are the level 1 index , and the 2nd column is the level 2 index , set them as such
X_test = X_test.set_index(['Instance', 'Time'])  
y_test = y_test.set_index(['Instance'])  

<>:2: SyntaxWarning: invalid escape sequence '\V'
<>:3: SyntaxWarning: invalid escape sequence '\V'
<>:2: SyntaxWarning: invalid escape sequence '\V'
<>:3: SyntaxWarning: invalid escape sequence '\V'
C:\Users\gwiku\AppData\Local\Temp\ipykernel_34596\2234204653.py:2: SyntaxWarning: invalid escape sequence '\V'
  X_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\X_test.csv')
C:\Users\gwiku\AppData\Local\Temp\ipykernel_34596\2234204653.py:3: SyntaxWarning: invalid escape sequence '\V'
  y_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\y_test.csv')


In [3]:
X_test

Open      High       Low     Close    Volume
Instance Time                                                  
0        0     0.210031  0.394983  0.000000  0.371473  0.517012
         1     0.570534  1.000000  0.434168  0.457681  0.021450
         2     0.437304  0.437304  0.067397  0.297805  0.118713
         3     0.506270  0.705330  0.429466  0.551723  0.000000
         4     0.626960  0.974921  0.605014  0.846395  0.087648
...                 ...       ...       ...       ...       ...
557      2     0.171173  0.513513  0.162162  0.360362  0.822514
         3     0.297299  0.630631  0.297299  0.468469  0.328470
         4     0.504505  0.720720  0.396398  0.675675  0.397136
         5     0.567568  0.639642  0.000000  0.396398  0.000000
         6     0.414417  0.846849  0.324325  0.684686  0.490831

[14385 rows x 5 columns]

In [4]:
y_test

,Pattern
Instance,
0,1
1,3
2,1
3,4
4,0
...,...
553,2
554,1
555,0


In [5]:
# max min and avg lengths of Instance index sets in X_test
max_len = 0
min_len = 100000
sum_len = 0
for i in range(1, 538):
    len_i = len(X_test.loc[i])
    sum_len += len_i
    if len_i > max_len:
        max_len = len_i
    if len_i < min_len:
        min_len = len_i
        
avg_len = sum_len / len(y_test)
max_len, min_len, avg_len

(275, 4, 24.933691756272403)

In [6]:
#  load test_patterns_with_symbols.csv and train_patterns_with_symbols.csv
test_patterns = pd.read_csv('Datasets/VanilaDataset/test_patterns_with_symbols.csv')
train_patterns = pd.read_csv('Datasets/VanilaDataset/train_patterns_with_symbols.csv')

# convert the Start and End columns to datetime
test_patterns['Start'] = pd.to_datetime(test_patterns['Start'])
test_patterns['End'] = pd.to_datetime(test_patterns['End'])
train_patterns['Start'] = pd.to_datetime(train_patterns['Start'])
train_patterns['End'] = pd.to_datetime(train_patterns['End'])


In [7]:
test_patterns

,Symbol,Chart Pattern,BullishBearish,Start,End,Industry
0,SMG,"Triangle, symmetrical",1,2021-10-06,2021-10-21,Chemical (Basic)
1,CLF,Head-and-shoulders top,0,2024-03-01,2024-03-13,Metals and Mining (Div.)
2,CR,"Triangle, symmetrical",0,2023-07-28,2023-09-28,Diversified Co.
3,IT,"Double Bottom, Adam and Adam",0,2022-10-13,2022-10-21,IT Services
4,JAZZ,"Double Top, Adam and Adam",-1,2022-07-07,2022-07-28,Biotechnology
...,...,...,...,...,...,...
554,K,"Double Bottom, Eve and Adam",0,2024-07-09,2024-07-15,Food Processing
555,ADM,"Triangle, symmetrical",1,2020-10-23,2020-12-02,Food Processing
556,HQY,"Double Top, Adam and Adam",0,2024-07-17,2024-07-26,Healthcare Information
557,LZB,"Double Bottom, Eve and Adam",1,2021-12-01,2021-12-20,Furn/Home Furnishings


In [8]:
test_patterns.loc[3, 'Symbol'] == train_patterns.loc[3, 'Symbol']

False

In [9]:
drop_count = 0
for i in range(len(test_patterns)):
    for j in range(len(train_patterns)):
        # Check if the time intervals intersect and belong to the same symbol
        if (
            test_patterns.loc[i, 'Start'] < train_patterns.loc[j, 'End'] and
            test_patterns.loc[i, 'End'] > train_patterns.loc[j, 'Start'] and
            test_patterns.loc[i, 'Symbol'] == train_patterns.loc[j, 'Symbol']
        ):
            # Calculate the intersection range
            intersection_start = max(test_patterns.loc[i, 'Start'], train_patterns.loc[j, 'Start'])
            intersection_end = min(test_patterns.loc[i, 'End'], train_patterns.loc[j, 'End'])
            intersection_length = (intersection_end - intersection_start).days  # Length in days

            # Calculate train segment length
            train_length = (train_patterns.loc[j, 'End'] - train_patterns.loc[j, 'Start']).days

            # Calculate the percentage of interaction
            interaction_percentage = (intersection_length / train_length) * 100

            print('Intersection Percentage:', interaction_percentage)

            # Drop the test row if interaction percentage > 60%
            if interaction_percentage > 60:
                print('Dropping test row with index:', i)
                test_patterns = test_patterns.drop(i)
                drop_count += 1
                break
            
print ('Dropped row count:', drop_count)


Intersection Percentage: 76.47058823529412
Dropping test row with index: 20
Intersection Percentage: 100.0
Dropping test row with index: 24
Intersection Percentage: 0.46728971962616817
Intersection Percentage: 100.0
Dropping test row with index: 55
Intersection Percentage: 100.0
Dropping test row with index: 72
Intersection Percentage: 10.204081632653061
Intersection Percentage: 10.204081632653061
Intersection Percentage: 10.204081632653061
Intersection Percentage: 87.5
Dropping test row with index: 79
Intersection Percentage: 100.0
Dropping test row with index: 107
Intersection Percentage: 92.3076923076923
Dropping test row with index: 113
Intersection Percentage: 100.0
Dropping test row with index: 116
Intersection Percentage: 2.898550724637681
Intersection Percentage: 100.0
Dropping test row with index: 155
Intersection Percentage: 100.0
Dropping test row with index: 160
Intersection Percentage: 95.23809523809523
Dropping test row with index: 161
Intersection Percentage: 100.0
Dropp

In [10]:
len(test_patterns)

529

In [11]:
new_combined_test_ohcl_df = []  # Initialize as a list to collect rows

extend_length = 50  # Number of days to extend the pattern

for index, row in test_patterns.iterrows():
    current_pattern_start_date = row['Start']
    current_pattern_end_date = row['End']
    
    # Add and subtract 300 days to get the new start and end dates
    new_pattern_end_date = current_pattern_end_date + pd.DateOffset(days=extend_length)
    new_pattern_start_date = current_pattern_start_date - pd.DateOffset(days=extend_length)
    
    print(f"Processing row {index + 1}/{len(test_patterns)}: Current pattern start: {current_pattern_start_date}, end: {current_pattern_end_date}", end="\r")
    
    for j in range(len(train_patterns)):
        train_start = train_patterns.loc[j, 'Start']
        train_end = train_patterns.loc[j, 'End']
        train_length = (train_end - train_start).days  # Train segment length in days
        
        # Calculate allowed intersection length (60%)
        allowed_intersection = train_length * 0.6
        
        # if the new pattern intersects with the train pattern
        if (
            new_pattern_start_date < train_end and
            new_pattern_end_date > train_start and
            row['Symbol'] == train_patterns.loc[j, 'Symbol']
        ):
            # Calculate the intersection range
            intersection_start = max(new_pattern_start_date, train_start)
            intersection_end = min(new_pattern_end_date, train_end)
            intersection_length = (intersection_end - intersection_start).days
            
            print(
                f"Checking train pattern {j + 1}/{len(train_patterns)}: Train start: {train_start}, end: {train_end}, intersection: {intersection_length} days",
                end="\r"
            )
            
            inter_perc = (intersection_length / train_length) * 100
            
            # if the intersection length is greater than 60% of the train pattern length
            if intersection_length > allowed_intersection:
                incremental_update_start_date = current_pattern_start_date
                incremental_update_end_date = current_pattern_end_date
                incr_intercal_length = 0
                
                while incr_intercal_length < allowed_intersection:
                    # change the start and end dates of the section by 1 day
                    incremental_update_start_date = incremental_update_start_date - pd.DateOffset(days=1)
                    incremental_update_end_date = incremental_update_end_date + pd.DateOffset(days=1)
                    
                    print(
                            f"Updated incrementally: Start: {incremental_update_start_date}, End: {incremental_update_end_date}",
                            end="\r"
                        )

                    # cheeck if the new section intersects with the train pattern
                    if incremental_update_start_date < train_end and incremental_update_end_date > train_start:
                        incr_intercal_start = max(incremental_update_start_date, train_start)
                        incr_intercal_end = min(incremental_update_end_date, train_end)
                        incr_intercal_length = (incr_intercal_end - incr_intercal_start).days
                        
                    # if the intersection length is greater than 60% of the train pattern length undo the last change and break
                    if incr_intercal_length > allowed_intersection:
                        incremental_update_start_date = incremental_update_start_date + pd.DateOffset(days=1)
                        incremental_update_end_date = incremental_update_end_date - pd.DateOffset(days=1)
                        break
                    # if somehow the new section is the same as the old 300 days changed section break
                    elif incremental_update_start_date == new_pattern_start_date and incremental_update_end_date == new_pattern_end_date:
                        break
                
                        
                new_pattern_start_date = incremental_update_start_date
                new_pattern_end_date = incremental_update_end_date
                    
    # add two new column test_sec_start and test_sec_end to row and append it to new_combined_test_ohcl_df
    row['test_sec_start'] = new_pattern_start_date
    row['test_sec_end'] = new_pattern_end_date
    
    new_combined_test_ohcl_df.append(row)

# Convert the result list to a DataFrame
new_combined_test_ohcl_df = pd.DataFrame(new_combined_test_ohcl_df)
print("\nProcessing complete.")

Processing row 559/529: Current pattern start: 2023-08-15 00:00:00, end: 2023-08-21 00:00:00, intersection: 1 dayss
Processing complete.


In [12]:
new_combined_test_ohcl_df

,Symbol,Chart Pattern,BullishBearish,Start,End,Industry,test_sec_start,test_sec_end
0,SMG,"Triangle, symmetrical",1,2021-10-06,2021-10-21,Chemical (Basic),2021-08-17,2021-12-10
1,CLF,Head-and-shoulders top,0,2024-03-01,2024-03-13,Metals and Mining (Div.),2024-01-11,2024-05-02
2,CR,"Triangle, symmetrical",0,2023-07-28,2023-09-28,Diversified Co.,2023-06-08,2023-11-17
3,IT,"Double Bottom, Adam and Adam",0,2022-10-13,2022-10-21,IT Services,2022-08-24,2022-12-10
4,JAZZ,"Double Top, Adam and Adam",-1,2022-07-07,2022-07-28,Biotechnology,2022-05-18,2022-09-16
...,...,...,...,...,...,...,...,...
554,K,"Double Bottom, Eve and Adam",0,2024-07-09,2024-07-15,Food Processing,2024-06-27,2024-07-27
555,ADM,"Triangle, symmetrical",1,2020-10-23,2020-12-02,Food Processing,2020-09-03,2021-01-21
556,HQY,"Double Top, Adam and Adam",0,2024-07-17,2024-07-26,Healthcare Information,2024-05-28,2024-09-14
557,LZB,"Double Bottom, Eve and Adam",1,2021-12-01,2021-12-20,Furn/Home Furnishings,2021-10-12,2022-02-08


In [13]:
origin_date = pd.to_datetime("2000-01-01")
test_sec_start_date = pd.to_datetime("2023-10-29")
test_sec_end_date = pd.to_datetime("2024-07-15")
test_start_date = pd.to_datetime("2024-03-01")
origin_date + (test_start_date- test_sec_start_date)

Timestamp('2000-05-04 00:00:00')

In [14]:
test_start_date -test_sec_start_date 

Timedelta('124 days 00:00:00')

In [15]:
test_new_combined_test_ohcl_df

NameError: name 'test_new_combined_test_ohcl_df' is not defined

In [ ]:
ohcl_data_current = pd.read_csv("Datasets/OHLS data/CLF.csv")

In [ ]:

ohcl_data_current["Date"] = pd.to_datetime(ohcl_data_current["Date"])
ohcl_data_current

In [ ]:
ohcl_data_current["Date"].iloc[0]

In [ ]:
origin_date

In [ ]:
# get the rows between start and end date sections of the row
ohcl_data_current = ohcl_data_current[
    (ohcl_data_current["Date"] >= test_sec_start_date) & (ohcl_data_current["Date"] <= test_sec_end_date)
]

ohcl_data_current.reset_index(drop=True, inplace=True)
ohcl_data_current

In [ ]:
date_diff=ohcl_data_current["Date"].iloc[0] - origin_date
date_diff

In [ ]:
date_diff= -87558
# convert date_diff to days
date_diff = date_diff.days

In [ ]:
date_diff= -87558
# convert date_diff to days
date_diff = date_diff.days
ohcl_data_current["Date"] = ohcl_data_current["Date"] - date_diff
ohcl_data_current

In [ ]:
row = new_combined_test_ohcl_df.iloc[1:2]
# convert the start and end dates to datetime
row['test_sec_start'] = pd.to_datetime(row['test_sec_start'])
row['test_sec_end'] = pd.to_datetime(row['test_sec_end'])   
row['Start'] = pd.to_datetime(row['Start'])
row['End'] = pd.to_datetime(row['End'])
row

In [ ]:
# calculate synthetic stream dates
synth_pattern_start = origin_date + (row["Start"] - row["test_sec_start"])
synth_pattern_end = origin_date + (row["End"] - row["test_sec_start"])

origin_date = origin_date + (row["test_sec_end"] - row["test_sec_start"])

print ((row["Start"] - row["test_sec_start"]) , (row["End"] - row["test_sec_start"]) , (row["test_sec_end"] - row["test_sec_start"]))
print (synth_pattern_start, synth_pattern_end, origin_date)

In [ ]:
# write a code to substract two given string dates and return the difference in days
def date_diff(date1, date2):
    date1 = pd.to_datetime(date1)
    date2 = pd.to_datetime(date2)
    
    return (date2 - date1).days

def date_sum(date, days):
    date = pd.to_datetime(date)
    
    return date + pd.DateOffset(days=days)

In [ ]:

#  loop through each row of new_combined_test_ohcl_df and create a new OHCL DataFrame with the corresponding OHCL data combined
OHCL_combined_df = pd.DataFrame()
test_stream_patterns = pd.DataFrame()
origin_date = pd.to_datetime("1800-01-01")

# get the 2nt row of new_combined_test_ohcl_df as the test_new_combined_test_ohcl_df DataFrame
test_new_combined_test_ohcl_df = new_combined_test_ohcl_df.iloc[0:6]

for i , row in new_combined_test_ohcl_df.iterrows():
    symbol = row["Symbol"]
    #  read the OHCL file
    ohcl_data_current = pd.read_csv("Datasets/OHLS data/" + symbol + ".csv")
    
    
    
    #  convert the Date column to datetime
    ohcl_data_current["Date"] = pd.to_datetime(ohcl_data_current["Date"])
    
    # get the rows between start and end date sections of the row
    ohcl_data_current = ohcl_data_current[
        (ohcl_data_current["Date"] >= row["test_sec_start"]) & (ohcl_data_current["Date"] <= row["test_sec_end"])
    ]
    
    ohcl_data_current.reset_index(drop=True, inplace=True)
    # print the ohcl_data_current dataframe and cleear it when the next one is printed
    print(ohcl_data_current)
    # If the DataFrame is empty, skip this iteration
    if ohcl_data_current.empty:
        print(f"No data for symbol {symbol} between {row['test_sec_start']} and {row['test_sec_end']}")
        continue
    
    # update all the dates in ohcl_data_current Date column to the corresponding synthetic stream dates starting from origin_date
    # to do that get the date difference between the first date in ohcl_data_current and the origin_date
    date_diff = ohcl_data_current["Date"].iloc[0] - origin_date
    
    # print ohcl_data_current["Date"].iloc[0] and origin_date and the difference 
    print("ohcl data current date:", str(ohcl_data_current["Date"].iloc[0]) , "origin date:", origin_date , "difference:", date_diff)
    
    # now reduce this date_diff from all the dates in the Date column
    ohcl_data_current["Date"] = ohcl_data_current["Date"] - date_diff
    # ohcl_data_current["Date"] = ohcl_data_current["Date"].apply(lambda d: pd.Timestamp(d) - date_diff)

    
    
    # print the ohcl_data_current dataframe and cleear it when the next one is printed
    print(ohcl_data_current)
    
    
    # combine the data to OHCL_combined_df
    OHCL_combined_df = pd.concat([OHCL_combined_df, ohcl_data_current])
    OHCL_combined_df.reset_index(drop=True, inplace=True)
    
    print("OHCL_combined_df", OHCL_combined_df)
    
    # calculate synthetic stream dates
    synth_pattern_start = origin_date + (row["Start"] - row["test_sec_start"])
    synth_pattern_end = origin_date + (row["End"] - row["test_sec_start"])
    
    # get the last date of ohcl_data_current and add 1 day to it to get the origin_date for the next iteration
    origin_date = ohcl_data_current["Date"].iloc[-1] + pd.DateOffset(days=1)
    
    print (row)
    print ((row["Start"] - row["test_sec_start"]) , (row["End"] - row["test_sec_start"]) , (row["test_sec_end"] - row["test_sec_start"]))
    print (synth_pattern_start, synth_pattern_end, origin_date)
    
    row_copy = row.copy()
    
    row_copy["Synth_Start"] = synth_pattern_start
    row_copy["Synth_End"] = synth_pattern_end
    
    # convert row_copy to a dataframe
    row_copy = pd.DataFrame(row_copy).T
    
    test_stream_patterns = pd.concat([test_stream_patterns, row_copy])
    
    print("test_stream_patterns", test_stream_patterns)
    
    test_stream_patterns.reset_index(drop=True, inplace=True)
    
    
    

In [ ]:
date_diff("2000-05-04", "2000-05-16")


In [ ]:
date_sum("2000-09-17", 1)

In [ ]:
OHCL_combined_df

In [ ]:
test_stream_patterns

In [ ]:
#  from test_stream_patterns drop Start , End , test_sec_start and test_sec_end columns
test_stream_patterns = test_stream_patterns.drop(columns=['Start', 'End', 'test_sec_start', 'test_sec_end'])
# rename Synth_Start and Synth_End columns to Start and End
test_stream_patterns.rename(columns={"Synth_Start": "Start", "Synth_End": "End"}, inplace=True)
test_stream_patterns

In [ ]:
import pandas as pd
import plotly.graph_objects as go


# Step 2: Prepare Data
OHCL_combined_df['Date'] = pd.to_datetime(OHCL_combined_df['Date'])

# Step 3: Plot Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=OHCL_combined_df['Date'],
                                     open=OHCL_combined_df['Open'],
                                     high=OHCL_combined_df['High'],
                                     low=OHCL_combined_df['Low'],
                                     close=OHCL_combined_df['Close'])])

# Step 4: Prepare test_stream_patterns

test_stream_patterns['Start'] = pd.to_datetime(test_stream_patterns['Start'])
test_stream_patterns['End'] = pd.to_datetime(test_stream_patterns['End'])

# Step 5: Mark Patterns
for index, row in test_stream_patterns.iterrows():

    start_date = row['Start']
    end_date = row['End']
    pattern = row['Chart Pattern']
    color = 'rgba(0, 0, 255, 0.2)' if row['BullishBearish'] == 'Bullish' else 'rgba(255, 0, 0, 0.2)'
    
    # Add shaded area for pattern range
    fig.add_shape(
        type="rect",
        xref="x",
        yref="paper",
        x0=start_date,
        y0=0,
        x1=end_date,
        y1=1,
        fillcolor=color,
        opacity=0.2,
        layer="below",
        line_width=0,
    )
    
    # Add annotation for pattern
    fig.add_annotation(x=start_date, y=OHCL_combined_df['High'].max(), text=pattern, showarrow=True, arrowhead=1)

# Step 6: Adjust Layout for Height and Zoom Controls
fig.update_layout(
    title='Test Stream Candlestick Chart with Pattern Annotations',
    height=800,
    autosize=True,
    yaxis={'fixedrange': False},
    xaxis={'fixedrange': False, 'rangeslider': {'visible': True}}
)

# Step 7: Display Chart
fig.show()
